In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import numpy as np
sns.set()


In [ ]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
print(df_test.columns)
print(df_test)

In [ ]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1)
df_test_id = pd.DataFrame()
df_test_id.index = df_test['PassengerId']
df_test = df_test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1)


In [ ]:
print(df_test_id)

In [ ]:
df['Sex'] = pd.factorize(df['Sex'])[0]
df_test['Sex'] = pd.factorize(df_test['Sex'])[0]
print(df)

In [ ]:
pd.get_dummies(df, prefix=['Embarked'])


In [ ]:

df['Pclass'] = df['Pclass'].apply(str)
df = pd.get_dummies(df, prefix=['PClass','Embarked'])
df_test['Pclass'] = df_test['Pclass'].apply(str)
df_test = pd.get_dummies(df_test, prefix=['PClass','Embarked'])

In [ ]:

df.boxplot(column =['Fare', 'Age'], grid = False)

In [ ]:
df['z_score']=scipy.stats.zscore(df['Fare'])
df.loc[df['z_score'].abs()>=3, 'Fare'] = df['Fare'].mean()
df = df.drop(['z_score'], axis=1)


In [ ]:
df.boxplot(column =['Fare', 'Age'], grid = False)

In [ ]:
df.isnull().sum(axis = 0)
df = df.fillna(df.mean())
df.isnull().sum(axis = 0)

In [ ]:
df_test.isnull().sum(axis = 0)
df_test = df_test.fillna(df.mean())
df_test.isnull().sum(axis = 0)

In [ ]:
df['Age_id'] = 0
df.loc[df['Age']<= 15,'Age_id'] = '~15'
df.loc[(df['Age'] > 15) & (df['Age'] >= 25),'Age_id'] = '15~25'
df.loc[(df['Age'] > 25) & (df['Age'] >= 35),'Age_id'] = '15~25'
df.loc[(df['Age'] > 35) & (df['Age'] >= 45),'Age_id'] = '15~25'
df.loc[(df['Age'] > 45) & (df['Age'] >= 55),'Age_id'] = '15~25'
df.loc[df['Age'] > 55,'Age_id'] = '55~'
print(df['Age_id'].head())

In [ ]:
df['Age']=(df['Age']-df['Age'].mean())/df['Age'].std()
df_test['Age']=(df_test['Age']-df_test['Age'].mean())/df_test['Age'].std()
df['Fare']=(df['Fare']-df['Fare'].mean())/df['Fare'].std()
df_test['Fare']=(df_test['Fare']-df_test['Fare'].mean())/df_test['Fare'].std()
print(df)

In [ ]:
df.plot.scatter(x = 'Age', y = 'Fare', s = 100);

In [ ]:
df.plot(y= 'Age', kind = 'line', use_index=True)

In [ ]:
def correlationmatrix(dataframe):
    corrMatrix = dataframe.corr()
    sns.heatmap(corrMatrix, annot=True)
    plt.show()

In [ ]:
correlationmatrix(df)

In [ ]:
def barchart_percentage(feature):
    survived_percentage = df[df['Survived'] ==1][feature].value_counts()/df[feature].value_counts()
    dead_percentage = df[df['Survived'] ==0][feature].value_counts()/df[feature].value_counts()
    df_analyse = pd.DataFrame([survived_percentage, dead_percentage])
    df_analyse.index = ['Survived', 'Dead']
    df_analyse.plot(kind='bar',stacked = False,figsize =(10,5))


In [ ]:
barchart_percentage('Sex')

In [ ]:
def barchart(feature):
    survived = df[df['Survived'] ==1][feature].value_counts()
    dead = df[df['Survived'] ==0][feature].value_counts()
    df_analyse = pd.DataFrame([survived, dead])
    df_analyse.index = ['Survived', 'Dead']
    df_analyse.plot(kind='bar',stacked = False,figsize =(10,5))
    
    

In [ ]:
barchart('Age_id')

In [ ]:
target_column = 'Survived' 
predictors = list(set(list(df.columns))-set([target_column, 'Age_id']))

print(target_column)
print(predictors)
predictors_test = list(set(list(df.columns))-set(['Age_id']))

In [ ]:
X = df[predictors].values
y = df[target_column].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

In [ ]:
X_test_set =df_test[predictors].values

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(8,8), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

In [ ]:

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)
predict_test_set = mlp.predict(X_test_set)
df_test_id['Survived'] = 0
df_test_id['Survived'] = predict_test_set
df_test_id.to_csv('predictions.csv')
print(df_test_id)

In [ ]:
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

In [ ]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))